# Dlaczego wasz model jest lepszy od innych? 


Otrzymaliśmy 92% trafności na danych testowych. Uważamy, że model jest wystarczająco dobry, żeby móc go zaimplementować

# Jak to sprawdziliście? 


Model przetrenowaliśmy na oczyszczonych danych (9826 pomiarów). Zawsze otrzymaliśmy skuteczność około 91%

# Dlaczego podjęliście takie a nie inne decyzje?


Chcieliśmy uprościć problem jak najbardziej. Dlatego podczas oczyszczania danych usunęliśmy wszystkie wiersze, które zawierały NaNy - błędne pomiary nie powinny być interpretowane przez model. Następnie wybraliśmy tylko wartości określone w kolumnach jako średnia (mean) lub odchylenie standardowe (std) - czyli najważniejsze dla nas wartości. Dzięki temu zmnieszyliśmy wymiarowość danych do 66 cech. Żeby sieć nie uczyła się też na danych, które zostały błędnie odczytane, obliczyliśmy dla każdej osoby (subject) w ramach każdej aktywności, wartości średnie dla wspomnianych wcześniej cech. Dało to nam 160 pomiarów treningowych. To wystarczyło sieci neuronowej na bardzo efektywną naukę.

Użyliśmy funkcji aktywacji softmax na końcu ze względu na jej charakter. Zwraca nam ona listę prawdopodobieństw, która sumuje się do 1. Można dzięki temu interpretować, które aktywności są najbardziej podobne do danego pomiaru, a które mniej

Kryteria oceny:
1. Czyszczenie danych i redukcja wymiarowości. (10 pkt)*
2. Dokładność modelu. (25 pkt)
3. Zrozumienie i interpretacja wyników. (35 pkt)*
4. Prezentacja i użyteczność biznesowa. (20 pkt)

## Wstępna analiza

Wczytujemy plik csv używając biblioteki pandas jako DataFrame, upewniamy się że widoczne będą wszystkie kolumny z danymi, analizujemy wielkość i naturę zbioru danych.

In [1]:
import pandas as pd
df = pd.read_csv('data/final_train.csv')
pd.set_option('display.max_columns', None)

In [2]:
df.shape

(11086, 573)

In [3]:
df.sample(6)

,Unnamed: 0,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16","fBodyAcc-bandsEnergy()-1,16.1","fBodyAcc-bandsEnergy()-1,16.2","fBodyAcc-bandsEnergy()-1,24","fBodyAcc-bandsEnergy()-1,24.1","fBodyAcc-bandsEnergy()-1,24.2","fBodyAcc-bandsEnergy()-1,8","fBodyAcc-bandsEnergy()-1,8.1","fBodyAcc-bandsEnergy()-1,8.2","fBodyAcc-bandsEnergy()-17,24","fBodyAcc-bandsEnergy()-17,24.1","fBodyAcc-bandsEnergy()-17,24.2","fBodyAcc-bandsEnergy()-17,32","fBodyAcc-bandsEnergy()-17,32.1","fBodyAcc-bandsEnergy()-17,32.2","fBodyAcc-bandsEnergy()-25,32","fBodyAcc-bandsEnergy()-25,32.1","fBodyAcc-bandsEnergy()-25,32.2","fBodyAcc-bandsEnergy()-25,48","fBodyAcc-bandsEnergy()-25,48.1","fBodyAcc-bandsEnergy()-25,48.2","fBodyAcc-bandsEnergy()-33,40","fBodyAcc-bandsEnergy()-33,40.1","fBodyAcc-bandsEnergy()-33,40.2","fBodyAcc-bandsEnergy()-33,48","fBodyAcc-bandsEnergy()-33,48.1","fBodyAcc-bandsEnergy()-33,48.2","fBodyAcc-bandsEnergy()-41,48","fBodyAcc-bandsEnergy()-41,48.1","fBodyAcc-bandsEnergy()-41,48.2","fBodyAcc-bandsEnergy()-49,56","fBodyAcc-bandsEnergy()-49,56.1","fBodyAcc-bandsEnergy()-49,56.2","fBodyAcc-bandsEnergy()-49,64","fBodyAcc-bandsEnergy()-49,64.1","fBodyAcc-bandsEnergy()-49,64.2","fBodyAcc-bandsEnergy()-57,64","fBodyAcc-bandsEnergy()-57,64.1","fBodyAcc-bandsEnergy()-57,64.2","fBodyAcc-bandsEnergy()-9,16","fBodyAcc-bandsEnergy()-9,16.1","fBodyAcc-bandsEnergy()-9,16.2",fBodyAcc-energy()-X,fBodyAcc-energy()-Y,fBodyAcc-energy()-Z,fBodyAcc-entropy()-X,fBodyAcc-entropy()-Y,fBodyAcc-entropy()-Z,fBodyAcc-iqr()-X,fBodyAcc-iqr()-Y,fBodyAcc-iqr()-Z,fBodyAcc-kurtosis()-X,fBodyAcc-kurtosis()-Y,fBodyAcc-kurtosis()-Z,fBodyAcc-mad()-X,fBodyAcc-mad()-Y,fBodyAcc-mad()-Z,fBodyAcc-main()-X,fBodyAcc-main()-Y,fBodyAcc-main()-Z,fBodyAcc-max()-X,fBodyAcc-max()-Y,fBodyAcc-max()-Z,fBodyAcc-maxInds-X,fBodyAcc-maxInds-Y,fBodyAcc-maxInds-Z,fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAcc-meanFreq()-X,fBodyAcc-meanFreq()-Y,fBodyAcc-meanFreq()-Z,fBodyAcc-min()-X,fBodyAcc-min()-Y,fBodyAcc-min()-Z,fBodyAcc-skewness()-X,fBodyAcc-skewness()-Y,fBodyAcc-skewness()-Z,fBodyAcc-sma(),fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,"fBodyAccJerk-bandsEnergy()-1,16","fBodyAccJerk-bandsEnergy()-1,16.1","fBodyAccJerk-bandsEnergy()-1,16.2","fBodyAccJerk-bandsEnergy()-1,24","fBodyAccJerk-bandsEnergy()-1,24.1","fBodyAccJerk-bandsEnergy()-1,24.2","fBodyAccJerk-bandsEnergy()-1,8","fBodyAccJerk-bandsEnergy()-1,8.1","fBodyAccJerk-bandsEnergy()-1,8.2","fBodyAccJerk-bandsEnergy()-17,24","fBodyAccJerk-bandsEnergy()-17,24.1","fBodyAccJerk-bandsEnergy()-17,24.2","fBodyAccJerk-bandsEnergy()-17,32","fBodyAccJerk-bandsEnergy()-17,32.1","fBodyAccJerk-bandsEnergy()-17,32.2","fBodyAccJerk-bandsEnergy()-25,32","fBodyAccJerk-bandsEnergy()-25,32.1","fBodyAccJerk-bandsEnergy()-25,32.2","fBodyAccJerk-bandsEnergy()-25,48","fBodyAccJerk-bandsEnergy()-25,48.1","fBodyAccJerk-bandsEnergy()-25,48.2","fBodyAccJerk-bandsEnergy()-33,40","fBodyAccJerk-bandsEnergy()-33,40.1","fBodyAccJerk-bandsEnergy()-33,40.2","fBodyAccJerk-bandsEnergy()-33,48","fBodyAccJerk-bandsEnergy()-33,48.1","fBodyAccJerk-bandsEnergy()-33,48.2","fBodyAccJerk-bandsEnergy()-41,48","fBodyAccJerk-bandsEnergy()-41,48.1","fBodyAccJerk-bandsEnergy()-41,48.2","fBodyAccJerk-bandsEnergy()-49,56","fBodyAccJerk-bandsEnergy()-49,56.1","fBodyAccJerk-bandsEnergy()-49,56.2","fBodyAccJerk-bandsEnergy()-49,64","fBodyAccJerk-bandsEnergy()-49,64.1","fBodyAccJerk-bandsEnergy()-49,64.2","fBodyAccJerk-bandsEnergy()-57,64","fBodyAccJerk-bandsEnergy()-57,64.1","fBodyAccJerk-bandsEnergy()-57,64.2","fBodyAccJerk-bandsEnergy()-9,16","fBodyAccJerk-bandsEnergy()-9,16.1","fBodyAccJerk-bandsEnergy()-9,16.2",fBodyAccJerk-energy()-X,fBodyAccJerk-energy()-Y,fBodyAccJerk-energy()-Z,fBodyAccJerk-entropy()-X,fBodyAccJerk-entropy()-Y,fBodyAccJerk-entropy(

W trakcie analizy dochodzimy do kilku wniosków, mianowicie:

In [4]:
pd.unique(df['Activity'])

array(['STANDING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING', 'SITTING',
       'WALKING_DOWNSTAIRS'], dtype=object)

1. Mamy 6 aktywności, które nasz model ma rozpoznawać,

In [5]:
df.subject.nunique()

30

2. W badaniu brało udział 30 uczestników,

3. Do trenowania modelu klasyfikacji definitywnie nie potrzebujemy wszystkich cech. Skupiamy się na tych, które zawierają średnią i odchylenie standardowe.

## Oczyszczanie danych

Zanim zaczniemy czyścić zbiór, wyciągamy z niego dane o uczestnikach i aktywnościach, to przyda nam się potem.

In [6]:
subject = df['subject']
activity = df['Activity']

Na początku wybieramy wszystkie kolumny zawierające średnią lub odchylenie standardowe wszystkich z cech.

In [7]:
mean_cols = [col for col in df.columns if 'mean' in col or 'Mean' in col]
std_cols = [col for col in df.columns if 'std' in col or 'Std' in col]

df = pd.concat([df[std_cols], df[mean_cols]], axis = 1)
df.shape

(11086, 87)

In [8]:
573-87, (573-87)/573

(486, 0.8481675392670157)

W ten sposób udało nam się pozbyć 486 cech, co stanowi prawie 85% całego zbioru.

Aktualnie nasz zbiór wygląda tak:

In [9]:
df.sample(4)

,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAcc-meanFreq()-X,fBodyAcc-meanFreq()-Y,fBodyAcc-meanFreq()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccJerk-meanFreq()-X,fBodyAccJerk-meanFreq()-Y,fBodyAccJerk-meanFreq()-Z,fBodyAccMag-mean(),fBodyAccMag-meanFreq(),fBodyBodyAccJerkMag-mean(),fBodyBodyAccJerkMag-meanFreq(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-meanFreq(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,fBodyGyro-meanFreq()-X,fBodyGyro-meanFreq()-Y,fBodyGyro-meanFreq()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
6646,-1.260664,-3.766730,3.381222,-6.546935,3.138447,1.136508,-3.559139,-2.788984,-6.083270,4.651769,1.962489,0.645767,3.615101,5.657498,3.691739,1.614544,4.508855,-3.227274,3.088312,3.444910,-2.607666,4.088879,1.409021,0.634290,-3.610477,-0.413330,0.855758,-2.251233,3.999339,3.627962,1.204948,3.943105,-3.021398,-1.567078,-0.882981,-2.489903,-1.971442,0.169322,-0.576820,7.888888,-8.165288,-7.187325,-0.200916,7.424041,-4.120972,3.573755,-1.905890,2.718808,-1.608246,-2.844121,-5.441640,6.932777,1.736199,4.603602,3.363426,3.311555,1.046295,-0.867452,0.073342,2.049136,2.246265,1.950773,2.441043,-2.005823,0.728023,2.074052,0.886770,3.991480,-1.104727,2.815380,0.108821,3.488897,1.788999,3.567289,4.706531,-2.521101,3.474765,-1.120686,1.099589,1.344511,-0.201598,1.760316,-1.722387,2.581667,3.041814,5.975428,-0.770148
10630,0.770361,-1.415377,4.154967,-4.065111,0.378903,2.323065,-1.052975,-3.004470,-6.023437,4.826976,3.253739,0.803673,4.685226,6.335615,3.874086,3.327267,6.283645,-0.630992,3.893686,2.363531,1.326861,5.299794,5.497756,1.771050,-4.121929,-0.582748,0.852488,-2.635729,5.647951,3.693231,1.416013,4.035716,1.670448,-1.570927,-0.464197,-2.440953,-1.853011,0.100345,1.564158,4.674324,-8.600804,-7.605688,-0.990099,6.414801,-4.009435,4.366929,-1.918147,4.652303,1.652083,-1.666396,-5.017214,7.364388,1.035392,7.384655,4.339895,3.335323,1.905267,-0.346052,0.855000,1.621775,2.903819,1.477599,2.528643,-5.472093,0.704576,1.401599,0.728453,3.935683,-0.941018,2.824064,0.788724,3.543405,1.855110,5.725129,3.621983,-2.049508,3.110699,-2.206660,1.119166,0.484326,2.339909,2.661111,0.316838,2.371594,2.036288,3.027965,-0.933337
7314,-0.284085,-2.882954,3.925613,-5.261620,2.082163,1.978638,-1.678040,-2.903099,-6.052033,4.746901,2.268739,0.719943,4.209043,5.963459,3.758300,2.803582,5.423837,-2.310950,3.624854,2.927659,0.216474,4.313949,3.498119,1.294625,-3.561727,-0.522402,0.853818,-2.481992,4.861632,3.674527,1.409776,4.087511,0.346333,-1.571852,-0.854279,-2.442750,-1.979659,0.360239,1.813549,2.168048,-11.979858,-7.340165,-0.474370,6.752840,-3.944474,3.660772,-1.929672,3.709547,-0

Dodatkowo zdecydowaliśmy się pozbyć kolumn, które zawierają słowa kluczowe: 'angle', 'energy' i 'meanFreq'

In [10]:
angle_cols = [col for col in df.columns if 'angle' in col or 'Angle' in col]
energy_cols = [col for col in df.columns if 'energy' in col or 'Energy' in col]
meanFreq_cols = [col for col in df.columns if 'meanFreq' in col]

drop_cols = angle_cols + energy_cols + meanFreq_cols
df = df.drop(drop_cols, axis=1)

In [11]:
df.shape

(11086, 66)

Ostatecznie nasz zbiór ma 66 cech.

Dodajemy kolumny uczestnika i aktywności.

In [12]:
df.insert(0, 'Activity', activity)
df.insert(0, 'Subject', subject)
df

,Subject,Activity,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccMag-mean(),fBodyBodyAccJerkMag-mean(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroMag-mean(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
0,30,STANDING,0.756296,-1.586674,4.141758,-4.176057,0.585073,2.245282,-1.000675,-2.979941,-6.033122,4.781842,2.565681,0.795967,4.632040,6.325173,3.860434,3.289566,6.187228,-0.830160,3.828060,2.470457,1.326475,4.699153,5.242485,1.709530,-4.047928,-0.567443,0.852718,-2.580826,5.308319,3.741996,1.394575,4.017211,1.669987,-7.592399,-0.928158,6.451225,4.527266,1.396293,-1.777539,7.292838,3.333295,-0.417207,1.684650,1.657815,2.521464,-5.258573,3.992989,-0.873160,2.846826,0.765912,3.545092,1.740686,5.620371,3.652962,-2.029314,3.154375,-2.118152,1.094581,0.533126,2.227442,2.599121,-0.003606,2.246320,2.861260,3.815614,-0.928676
1,9,WALKING_UPSTAIRS,-0.537596,-2.784092,3.677794,-5.817531,1.293797,1.616815,-2.166122,-2.906000,-6.062388,4.653310,1.148811,0.660377,3.794268,5.880477,3.775318,2.227671,5.072585,-1.472494,3.399457,2.920422,-0.530696,3.429869,2.133231,0.925902,-3.601759,-0.491527,0.853675,-2.404219,4.198657,3.720622,1.202847,4.050714,-0.544652,-7.291138,-0.623965,7.141304,3.393216,0.616804,-2.409713,5.965342,3.322065,-0.649755,1.949084,2.082325,2.470419,-3.498200,3.990053,-0.902530,2.853683,1.396476,3.512594,1.801041,4.410433,4.357823,-2.404910,3.088970,-1.823719,1.156491,0.567243,1.192297,2.187126,-1.502168,2.488702,2.391994,5.884982,-0.822617
2,28,LAYING,0.755296,-1.283612,4.217478,-4.063730,0.303414,2.333802,-0.806943,-3.011841,-6.017739,4.845153,3.302522,0.807615,4.705391,6.331799,3.883619,3.456453,6.284308,-0.557388,3.904492,2.332787,1.623716,5.340579,5.605033,1.789814,-4.134095,-0.587106,0.852456,-2.656659,5.780229,3.621391,1.430272,4.146776,2.024442,-7.605039,-1.026274,6.347764,4.652675,1.747315,-1.646547,7.496709,3.335892,-0.329539,1.598720,1.465659,2.531127,-5.514689,3.922214,-0.946887,2.876148,0.782550,3.540643,1.812028,5.735402,3.573648,-2.063205,3.135899,-2.219777,1.116531,0.496151,2.328004,2.666988,0.338996,4.616200,1.336379,0.032001,-0.940610
3,25,WALKING,0.149588,-2.796041,3.922191,-4.776575,1.763192,1.876670,-1.633627,-2.920310,-6.048324,4.737144,1.710101,0.746790,4.400935,6.119588,3.769096,2.769627,5.735377,-1.815135,3.567625,2.780547,0.387614,3.929503,4.154581,1.480092,-3.555616,-0.541816,0.853448,-2.497160,4.913941,3.698913,1.319806,4.098944,0.550414,-7.456240,-0.551743,6.816364,4.004257,0.307571,-2.173669,6.682904,3.326199,-0.521936,1.782375,1.919775,2.504932,-4.642801,4.011555,-0.971376,2.884066,0.826285,3.501259,1.645810,4.927229,4.063273,-1.966041,3.131273,-2.128185,1.109825,0.344011,2.133466,2.319992,-0.632525,2.254465,2.830937,3.575867,-0.866937
4,28,LAYING,0.755296,-1.283612,4.217478,-4.063730,0.303414,2.333802,-0.806943,-3.011841,-6.017739,4.845153,3.302522,0

Ostatecznie zdecydowaliśmy się zgrupować wszystkich uczestników badania razem z aktywnościami, w których brali udział

In [13]:
grouped = df.groupby(['Subject', 'Activity'])

Wyciągamy średnią z wszystkich cech, dla każdego uczestnika względem każdej z badanych aktywności.

In [14]:
df = grouped.mean().reset_index()

I tak prezentują się nasze oczyszczone dane:

In [15]:
df.head(10)

,Subject,Activity,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccMag-mean(),fBodyBodyAccJerkMag-mean(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroMag-mean(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
0,1,LAYING,0.742534,-1.352281,4.165208,-4.048067,0.307285,2.339434,-0.946739,-3.012469,-6.019987,4.825410,3.137130,0.810149,4.673888,6.328449,3.878952,3.355537,6.293759,-0.564716,3.909856,2.329932,1.456830,5.198991,5.663404,1.759351,-4.114366,-0.587992,0.852487,-2.648185,5.641730,3.629403,1.361852,3.957349,1.825434,-7.605127,-1.012578,6.383025,4.661432,1.734590,-1.636783,7.438892,3.335956,-0.336816,1.619584,1.506020,2.532155,-5.439476,3.975365,-0.926304,2.909707,0.759630,3.539508,1.800647,5.741904,3.623470,-2.085956,3.151520,-2.329827,1.116704,0.467913,2.408384,2.666890,0.251195,4.606420,0.958898,2.003782,-0.933114
1,1,SITTING,0.792379,-1.317112,4.209662,-4.026415,0.266434,2.362534,-0.825279,-3.015763,-6.015538,4.833118,3.293136,0.810766,4.661738,6.344424,3.881932,3.449210,6.312880,-0.523025,3.926646,2.312788,1.614337,5.340870,5.691894,1.750593,-4.169973,-0.590864,0.852425,-2.667632,5.706143,3.709399,1.380383,4.094929,2.013258,-7.613506,-1.026653,6.340513,4.686398,1.790327,-1.610813,7.509218,3.336345,-0.314262,1.604770,1.456749,2.533228,-5.438943,3.988319,-0.943407,2.876509,0.772236,3.542796,1.791074,5.773077,3.562669,-2.023633,3.152543,-2.058776,1.120879,0.489642,2.256600,2.688406,0.276187,2.463385,2.122535,2.552489,-0.942262
2,1,STANDING,0.802141,-1.296172,4.213388,-4.021199,0.269555,2.364546,-0.804977,-3.016560,-6.015802,4.848482,3.320864,0.812218,4.703371,6.347711,3.883188,3.457751,6.317113,-0.526052,3.928125,2.310115,1.641992,5.366334,5.721486,1.788767,-4.166715,-0.591226,0.852428,-2.668376,5.813352,3.730624,1.394607,4.113227,2.046236,-7.615628,-1.029197,6.333663,4.691739,1.786385,-1.608289,7.524556,3.336380,-0.314046,1.588494,1.447581,2.533525,-5.517504,3.995074,-0.954695,2.878224,0.769484,3.540635,1.786922,5.773861,3.556871,-2.066622,3.141861,-2.196288,1.117609,0.495737,2.306182,2.687757,0.355008,2.297485,3.113136,3.942945,-0.943135
3,1,WALKING,-0.421209,-3.346157,3.827163,-5.742295,2.730468,1.611067,-2.143042,-2.846935,-6.101496,4.649844,1.875883,0.654746,4.139156,5.908111,3.725957,2.528747,5.016724,-2.724110,3.399913,3.182890,-0.564691,3.974289,1.761987,1.202327,-3.153534,-0.424454,0.854317,-2.204821,4.041325,3.665274,1.380540,4.063922,-0.581984,-7.291918,-0.360802,7.004477,3.185582,-0.836279,-2.402467,5.887284,3.316753,-0.887231,2.023027,2.025512,2.453120,-3.799711,3.977182,-0.940109,2.874388,0.835021,3.550443,1.866594,4.189866,4.348208,-2.052337,3.117138,-2.257727,1.119942,0.687839,2.163259,1.857786,-1.173680,2.310523,3.117683,4.308172,-0.823877
4,1,WALKING_DOWNSTAIRS,-0.937546,-3.193000,3.840883,-5.901125,2.468137,1.413464,-2.660484,-2.838354,-6.110421,4.662521,1.519395,0.